<a href="https://colab.research.google.com/github/mvmarcio/Econometria_Wooldridge_R/blob/main/Fama_MacBeth_Marcelo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Aplicação da Metodologia Fama-MacBeth no R**

Este script tem o objetivo de se aplicar o modelo de Fama-MacBeth. 

## **1. Importando os Pacotes Necessários**

In [ ]:
install.packages("quantmod")
install.packages("PerformanceAnalytics")
install.packages("TTR")
install.packages("repmis")
install.packages("zoo")
install.packages("sandwich")
install.packages("lmtest")
install.packages("readxl")
install.packages("openxlsx")
install.packages("stargezer")
install.packages("car")
install.packages("AER")
install.packages("lubridate")
install.packages("broom")

In [ ]:
library(quantmod)
library(PerformanceAnalytics)
library(TTR)
library(repmis)
# library(zoo)
library(sandwich)
library(lmtest)
library(readxl)
library(openxlsx)
#library(stargezer)
library(car)
library(AER)
library(psych)
library(tidyr)
library(tidyverse)
library(lubridate)
library(broom)


Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union




## **2. Carregando a Base de Dados**

In [ ]:
dados <- read_excel("base.xlsm", sheet = "Planilha5")

In [ ]:
tail(dados) #últimas linhas

In [ ]:
head(dados) #primeiras linhas

In [ ]:
# Estatísticas descritivas:
describe(dados, quant = c(.25, .75))

In [ ]:
# Retirando os valores faltantes:
dados <- na.omit(dados)

In [ ]:
head(dados)

In [ ]:
# Gerando série de retornos:

retorno <- ts(dados$retorno, frequency = 365, start = c(2015, 01, 02))

In [ ]:
# Gerando série do fluxo:
fluxo <- ts(dados$`Fluxo (Variável Dependente)`, frequency = 365, start = c(2015, 01, 2))

In [ ]:
head(retorno)

[1]  8.500083e-05  1.869427e-03 -2.030686e-03 -3.142833e-04  2.487521e-04
[6] -1.306841e-03

In [ ]:
# Gerando série de riscos:
risco <- ts(dados$`Risco     (Volatilidade)`, frequency = 365, start = c(2015, 01, 02))

In [ ]:
# Criando séries com as dummies:
ganho <- ts(dados$Ganho, frequency = 365, start = c(2015, 01, 02))
perda <- ts(dados$Perda, frequency = 365, start = c(2015, 01, 02))

In [ ]:
perda

A Time Series:
   [1] 1 0 1 1 1 1 1 0 0 1 0 1 1 0 0 1 1 1 1 1 1 1 1 0 1 0 1 0 1 1 1 0 1 0 0 0 1
  [38] 0 0 1 1 1 0 0 0 0 0 1 0 1 1 1 0 1 1 1 0 1 1 0 1 1 0 1 0 1 0 0 0 1 1 1 1 1
  [75] 0 0 0 1 1 1 0 0 1 1 1 1 0 1 1 1 1 0 0 1 0 0 1 0 0 1 1 0 1 1 0 1 1 1 1 1 1
 [112] 1 1 1 1 0 0 1 1 0 1 1 0 1 0 1 0 1 1 1 1 1 1 0 0 0 1 0 0 0 1 0 1 0 0 0 1 0
 [149] 0 1 1 0 1 0 1 0 1 0 1 0 0 0 1 1 1 1 0 0 1 0 1 1 1 1 0 1 1 1 0 1 1 1 0 0 0
 [186] 1 1 1 1 1 0 1 1 1 1 1 1 0 1 0 1 0 0 0 0 0 1 1 1 0 1 0 0 1 1 1 0 1 1 1 1 0
 [223] 1 1 1 1 1 0 1 1 1 1 1 1 0 1 0 1 1 1 0 1 1 1 1 1 1 1 0 1 0 0 1 0 1 1 1 0 1
 [260] 0 1 1 0 0 1 0 1 0 1 1 1 0 1 1 0 0 0 1 1 0 1 0 1 1 0 1 1 1 1 1 1 1 1 1 0 1
 [297] 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 1 1 0 1 0 1 0 1 1 1 1 0 1 0 1 1
 [334] 1 0 1 1 0 0 0 0 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 0
 [371] 0 1 1 1 0 1 1 1 1 0 1 1 0 0 0 0 1 1 1 0 1 0 0 1 0 1 1 0 0 0 1 0 1 0 1 1 1
 [408] 1 1 0 1 1 1 1 1 0 1 0 1 0 1 0 1 1 0 1 1 0 0 1 0 0 0 1 1 0 0 1 1 0 1 0 0 0
 [445] 0 1 0 1 0 0 1 0 1 0 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1
 [482] 1 1 0 0 0 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 0 1 0 0 1
 [519] 1 1 0 1 1 0 1 0 1 0 0 0 0 0 1 0 1 0 0 0 1 1 1 0 1 0 1 1 1 1 0 0 1 0 1 1 1
 [556] 1 1 1 0 0 1 0 1 0 0 0 1 1 0 0 1 1 1 0 0 1 1 1 1 1 1 0 0 1 1 0 1 0 0 0 1 0
 [593] 0 1 1 0 1 0 0 1 0 0 1 0 1 1 1 0 1 1 0 1 1 1 0 0 1 0 1 1 1 1 0 1 0 0 1 1 1
 [630] 1 1 1 0 1 1 1 1 0 0 1 1 1 1 0 0 1 0 0 1 1 1 1 1 1 0 0 1 1 1 0 1 0 0 0 0 1
 [667] 0 1 1 1 1 1 0 1 1 0 0 0 0 0 0 1 0 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1
 [704] 1 0 1 1 0 1 0 1 1 1 1 1 1 1 0 1 1 0 1 0 1 0 1 1 0 1 0 1 1 1 1 1 0 0 1 0 1
 [741] 0 1 1 1 1 0 0 1 0 1 0 0 1 1 1 1 1 0 0 0 1 0 0 0 1 1 0 1 0 1 1 0 1 1 1 0 1
 [778] 1 1 0 0 0 1 1 0 0 0 1 1 1 1 0 0 1 1 1 0 0 1 1 1 0 1 1 0 0 0 0 1 1 0 1 1 1
 [815] 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 0 1 1 1 1 1 0 0
 [852] 1 0 1 0 1 1 1 0 0 0 0 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 0 1 1 0 1 1
 [889] 0 1 0 0 1 1 1 1 1 1 0 1 1 1 1 1 0 0 1 1 1 1 1 0 1 1 0 1 0 1 1 1 1 1 0 1 1
 [926] 1 1 1 0 1 1 1 0 1 1 1 0 1 1 0 0 1 1 0 1 1 1 1 0 1 1 1 0 1 1 0 0 1 0 1 0 0
 [963] 1 1 1 1 1 1 0 0 1 1 1 1 1 1 0 0 1 1 0 1 1 1 1 1 1 0 0 1 1 0 1 1 1 1 1 0 0
[1000] 1 0 0 0 1 0 0 1 1 0 1 1 0 1 1 1 0 1 1 1 0 1 0 1 1 1 1 1 1 1 0 1 0 1 1 1 1
[1037] 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 0
[1074] 1 0 1 0 1 1 1 1 1 1 0 1 1 1 0 0 1 0 1 1 1 0 0 1 0 1 0 0 1 1 1 1 1 1 1 0 1
[1111] 1 1 0 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1
[1148] 1 1 0 0 0 1 1 0 1 1 1 1 1 0 1 1 1 1 1 0 0 1 1 0 1 1 1 1 1 0 1 1 0 1 0 1 1
[1185] 1 1 0 1 1 1 1 0 0 1 1 0 0 0 1 1 0 1 1 0 1 1 1 0 1 1 0 1 0 1 1 0 1 1 1 1 1
[1222] 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 0 1 1 1 1
[1259] 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 0 0
[1296] 0 1 1 1 0 1 1 0 1 0 1 0 1 0 1 1 1 1 0 1 1 1 1 0 1 1 1 1 0 1 1 0 0 0 1 1 1
[1333] 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 0 0 1
[1370] 1 1 1 1 1 1 1 0 1 1 0 1 0 1 1 1 1 0 1 0 0 1 1 1 1 0 1 0 1 1 1 1 1 0 1 1 1
[1407] 1 1 0 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1
[1444] 0 1 0 0 0 1 1 1 1 1 1 0 1 0 1 1 1 1 1 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1
[1481] 0 1 0 1 1 0 1 1 1 1 1 1 0 0 0 1 1 1 0 1 0 1 0 1 1 0 0 0 0 1 1 1 1 1 1 0 0
[1518] 1 1 1 1 1 0 1 0 0 1 0 0 1 1 1 0 0 1 1 1 1 1 0 1 1 0 1 1 1 0 1 0 0 0 1 1 1
[1555] 0 1 1 0 1 1 1 0 1 1 0 0 0 1 1 1 0 1 1 1 0 0 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1
[1592] 1 1 0 0 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 0 0 1 0 1 1 0 0 1 1 1 1 1 1 1 1
[1629] 0 1 0 1 0 1 1 1 0 1 0 0 0 1 0 0 0 1 1 0 0 0 0 0 1 0 0 1 1 0 1 1 1 0 1 1 0
[1666] 1 1 0 1 0 1 0 0 0 1 0 1 1 0 0 0 0 1 1 1 0 0 1 1 1 1 0 1 1 1 1 0 0 1 0 0 1
[1703] 0 0 1 1 0 1 1 0 1 1 1 1 1 0 1 1 1 1 0 0 1 1 0 1 1 0 1 0 1 0 1 1 1 1 1 1 0
[1740] 0 0 1 0 0 0 1 0 1 0 0 0 1 0 0 1 1 0 1 0 1 0 1 0 1 0 0 0 1 1 1 1 0 0 1 0 1
[1777] 1 1 1 0 1 1 1 0 1 1 1 0 0 0 1 1 1 1 1 0 1 1 1 1 1 1 0 1 0 1 0 0 0 0 0 1 1
[1814] 1 0 1 0 0

In [ ]:
# Gerando defasagens de 30 dias:
lretorno <- stats::lag(retorno, 30)
lrisco <- stats::lag(risco, 30)

## **Gerando o Modelo**

In [ ]:
modelo <- lm(fluxo ~ 0 + lrisco + lretorno + ganho + perda)

In [ ]:
summary(modelo)


Call:
lm(formula = fluxo ~ 0 + lrisco + lretorno + ganho + perda)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.22983 -0.00082 -0.00047  0.00061  1.43670 

Coefficients:
           Estimate Std. Error t value Pr(>|t|)  
lrisco   -0.1722088  0.1153084  -1.493   0.1354  
lretorno  0.0322368  0.0837709   0.385   0.7004  
ganho     0.0012237  0.0006785   1.804   0.0713 .
perda     0.0008341  0.0004294   1.943   0.0521 .
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.02338 on 8024 degrees of freedom
Multiple R-squared:  0.0008039,	Adjusted R-squared:  0.0003058 
F-statistic: 1.614 on 4 and 8024 DF,  p-value: 0.1677


In [ ]:
coefs_avg = modelo %>%
    summary(Mean.Estimators = mean(estimate), 
              Std.Estimators = sd(estimate), 
              Mean.Std.Err = mean(std.error),
              Mean.P.value = mean(p.value))

In [ ]:
coefs_avg



Call:
lm(formula = fluxo ~ 0 + lrisco + lretorno + ganho + perda)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.22983 -0.00082 -0.00047  0.00061  1.43670 

Coefficients:
           Estimate Std. Error t value Pr(>|t|)  
lrisco   -0.1722088  0.1153084  -1.493   0.1354  
lretorno  0.0322368  0.0837709   0.385   0.7004  
ganho     0.0012237  0.0006785   1.804   0.0713 .
perda     0.0008341  0.0004294   1.943   0.0521 .
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.02338 on 8024 degrees of freedom
Multiple R-squared:  0.0008039,	Adjusted R-squared:  0.0003058 
F-statistic: 1.614 on 4 and 8024 DF,  p-value: 0.1677


## **Análise dos Resultados**

As variáveis risco e retorno foram defasadas em 30 dias para capturar as entradas e saídas por conta deste período. 



Os coeficientes das *dummies* ganho e perda foram estatisticamente significativos a 10% em relação à variabilidade do fluxo do fundo e com o sinal negativo. Neste caso, em caso de ganhos e perdas houve uma maior entrada nos fundos em questão. 